In [1]:
import pandas as pd
import numpy as np
import gzip, json, re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [2]:
with gzip.open('meta_Digital_Music.json.gz', 'rb') as f:
    data = f.read()
    data = data.decode('utf-8')

In [3]:
songs_hengameh = pd.read_json(data, lines=True)

In [4]:
songs_hengameh.head(3)

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,[],,[],,Master Collection Volume One,"[B000002UEN, B000008LD5, B01J804JKE, 747403435...",,John Michael Talbot,[],"58,291 in CDs & Vinyl (","[B000002UEN, B000008LD5, 7474034352, B000008LD...","<img src=""https://images-na.ssl-images-amazon....",,,$18.99,0001377647,[],[],NaN
1,[],,[],,Hymns Collection: Hymns 1 &amp; 2,"[5558154950, B00014K5V4]",,Second Chapter of Acts,[],"93,164 in CDs & Vinyl (","[B000008KJ3, B000008KJ0, 5558154950, B000UN8KZ...","<img src=""https://images-na.ssl-images-amazon....",,,,0001529145,[],[],NaN
2,[],,[],,Early Works - Don Francisco,"[B00004RC05, B003H8F4NA, B003ZFVHPO, B003JMP1Z...",,Don Francisco,[],"875,825 in CDs & Vinyl (","[B003H8F4NA, B003ZFVHPO, B003JMP1ZK, B00004RC0...","<img src=""https://images-na.ssl-images-amazon....",,,,0001527134,[],[],NaN


In [5]:
songs_hengameh.columns

Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')

In [6]:
songs_hengameh.size

1412593

In [7]:
songs_hengameh.shape

(74347, 19)

In [8]:
songs_hengameh.info() #songs_hengameh.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74347 entries, 0 to 74346
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   category         74347 non-null  object
 1   tech1            74347 non-null  object
 2   description      74347 non-null  object
 3   fit              74347 non-null  object
 4   title            74347 non-null  object
 5   also_buy         74347 non-null  object
 6   tech2            74347 non-null  object
 7   brand            74347 non-null  object
 8   feature          74347 non-null  object
 9   rank             74347 non-null  object
 10  also_view        74347 non-null  object
 11  main_cat         74347 non-null  object
 12  similar_item     74347 non-null  object
 13  date             74347 non-null  object
 14  price            74347 non-null  object
 15  asin             74347 non-null  object
 16  imageURL         74347 non-null  object
 17  imageURLHighRes  74347 non-null

In [9]:
songs_hengameh.isna().sum()

category             0
tech1                0
description          0
fit                  0
title                0
also_buy             0
tech2                0
brand                0
feature              0
rank                 0
also_view            0
main_cat             0
similar_item         0
date                 0
price                0
asin                 0
imageURL             0
imageURLHighRes      0
details            534
dtype: int64

In [10]:
# empty_values = (songs_hengameh.eq('') | songs_hengameh.eq(' ') |\
#                     songs_hengameh.applymap(lambda x: isinstance(x, list) and len(x)==0) | \
#                     songs_hengameh.applymap(lambda x: isinstance(x, dict) and len(x)==0) | \
#                     songs_hengameh.isnull()).sum().sum()

songs_hengameh = songs_hengameh.applymap(lambda x: np.nan if ((type(x) == str and len(x) == 0) or\
                    (type(x) == list and len(x) == 0) or (type(x) == dict and len(x) == 0)) else x)
songs_hengameh.isnull().sum()

category           74340
tech1              74347
description        37821
fit                74347
title                677
also_buy           64960
tech2              74347
brand              14471
feature            74258
rank                5938
also_view          60758
main_cat               0
similar_item       74347
date               74342
price              27897
asin                   0
imageURL           51124
imageURLHighRes    51124
details             1540
dtype: int64

In [11]:
empty_columns = ['category', 'tech1', 'fit', 'tech2', 'feature', 'main_cat', 'similar_item', 'date', 'imageURL', 'imageURLHighRes', 'details']
songs_hengameh.drop(columns=empty_columns, inplace=True)

In [12]:
songs_hengameh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74347 entries, 0 to 74346
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  36526 non-null  object
 1   title        73670 non-null  object
 2   also_buy     9387 non-null   object
 3   brand        59876 non-null  object
 4   rank         68409 non-null  object
 5   also_view    13589 non-null  object
 6   price        46450 non-null  object
 7   asin         74347 non-null  object
dtypes: object(8)
memory usage: 4.5+ MB


In [13]:
songs_hengameh[3:6]

,description,title,also_buy,brand,rank,also_view,price,asin
3,NaN,So You Wanna Go Back to Egypt,"[B0000275QQ, 0001393774, 0001388312, B0016CP2G...",Keith Green,"203,263 in CDs & Vinyl (","[B00000I7JO, B0016CP2GS, 0001393774, B0000275Q...",$13.01,0001388703
4,[1. Losing Game 2. I Can't Wait 3. Didn't He S...,Early Works - Dallas Holm,"[B0002N4JP2, 0760131694, B00002EQ79, B00150K8J...",Dallas Holm,"399,269 in CDs & Vinyl (","[B0002N4JP2, 0760131694, B00150K8JC, B003MTXNV...",NaN,0001526146
5,NaN,The Glory of Love (Solo Piano),NaN,Michael Faircloth,"779,669 in CDs & Vinyl (",NaN,NaN,0005096421


In [14]:
songs_hengameh['description'][4]

["1. Losing Game 2. I Can't Wait 3. Didn't He Shine 4. Never Seen...Righteous... 5. A Broken Heart 6. Looking Back 7. Here We Are 8. I Saw The Lord 9. Jesus Is A River Of Love 10. Hittin' The Road 11. I've Never Been Out Of... 12. Jesus Gotta Hold Of My Life 13. Saved- Saved- Saved 14. What Will You Do? 15. Rise Again"]

In [15]:
songs_hengameh['title'][4]

'Early Works - Dallas Holm'

In [16]:
songs_hengameh['description'] = songs_hengameh['description'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
songs_hengameh['also_buy'] = songs_hengameh['also_buy'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
songs_hengameh['also_view'] = songs_hengameh['also_view'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [17]:
songs_hengameh['popular_asin'] = songs_hengameh['also_buy'].fillna('') + ' ' + songs_hengameh['also_view'].fillna('')

In [18]:
songs_hengameh['product_info'] = songs_hengameh['title'].fillna('') + ' ' + songs_hengameh['description'].fillna('') + ' ' + songs_hengameh['brand'].fillna('') 

In [19]:
songs_hengameh = songs_hengameh.drop(['description', 'brand', 'also_buy', 'also_view'],axis=1)
songs_hengameh.head(3)

,title,rank,price,asin,popular_asin,product_info
0,Master Collection Volume One,"58,291 in CDs & Vinyl (",$18.99,0001377647,"B000002UEN, B000008LD5, B01J804JKE, 7474034352...",Master Collection Volume One John Michael Talbot
1,Hymns Collection: Hymns 1 &amp; 2,"93,164 in CDs & Vinyl (",NaN,0001529145,"5558154950, B00014K5V4 B000008KJ3, B000008KJ0,...",Hymns Collection: Hymns 1 &amp; 2 Second Chap...
2,Early Works - Don Francisco,"875,825 in CDs & Vinyl (",NaN,0001527134,"B00004RC05, B003H8F4NA, B003ZFVHPO, B003JMP1ZK...",Early Works - Don Francisco Don Francisco


In [20]:
# songs_hengameh['also_buy'] = songs_hengameh['also_buy'].fillna('')
# buy_dummies = pd.get_dummies(songs_hengameh['also_buy'].astype(str).str.split(',').apply(pd.Series).stack(), prefix='buy').sum(level=0)
# songs_hengameh = pd.concat([songs_hengameh, buy_dummies], axis=1)

# songs_hengameh['also_view'] = songs_hengameh['also_view'].fillna('')
# view_dummies = pd.get_dummies(songs_hengameh['also_view'].astype(str).str.split(',').apply(pd.Series).stack(), prefix='view').sum(level=0)
# songs_hengameh = pd.concat([songs_hengameh, view_dummies], axis=1)

# brand_dummies = pd.get_dummies(df['brand'].fillna(''), prefix='brand', prefix_sep='')
# songs_hengameh = pd.concat([songs_hengameh, brand_dummies], axis=1)

In [21]:
songs_hengameh['price'] = songs_hengameh['price'].replace('[\$,]', '', regex=True)

In [22]:
songs_hengameh['price'][12]

'9600.61'

In [23]:
songs_hengameh['price'] = pd.to_numeric(songs_hengameh['price'], errors='coerce')

In [24]:
songs_hengameh['price'].fillna(songs_hengameh['price'].median(), inplace=True)

In [25]:
songs_hengameh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74347 entries, 0 to 74346
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         73670 non-null  object 
 1   rank          68409 non-null  object 
 2   price         74347 non-null  float64
 3   asin          74347 non-null  object 
 4   popular_asin  74347 non-null  object 
 5   product_info  74347 non-null  object 
dtypes: float64(1), object(5)
memory usage: 3.4+ MB


In [26]:
songs_hengameh_main = songs_hengameh.copy()

In [27]:
songs_hengameh.applymap(lambda x: 'http' in str(x)).sum()

title            0
rank             0
price            0
asin             0
popular_asin     0
product_info    27
dtype: int64

In [28]:
# songs_hengameh.product_info[110:115]

In [29]:
songs_hengameh.title[110:115]

110                                      Pure Motivation
111    <span class="a-size-medium a-color-secondary a...
112     Tai Chi Meditation 2: Eight Direction Perception
113           Tai Chi Meditation 1: Life Force Breathing
114                                Sounds Fun Phonics CD
Name: title, dtype: object

In [30]:
emoji_pattern = re.compile('['
    '\U0001F1E0-\U0001F1FF'  # flags (iOS)
    '\U0001F300-\U0001F5FF'  # symbols & pictographs
    '\U0001F600-\U0001F64F'  # emoticons
    '\U0001F680-\U0001F6FF'  # transport & map symbols
    '\U0001F700-\U0001F77F'  # alchemical symbols
    '\U0001F780-\U0001F7FF'  # Geometric Shapes Extended
    '\U0001F800-\U0001F8FF'  # Supplemental Arrows-C
    '\U0001F900-\U0001F9FF'  # Supplemental Symbols and Pictographs
    '\U0001FA00-\U0001FA6F'  # Chess Symbols
    '\U0001FA70-\U0001FAFF'  # Symbols and Pictographs Extended-A
    '\U00002702-\U000027B0'  # Dingbats
    '\U000024C2-\U0001F251'
    ']+')

In [31]:
for col in songs_hengameh.columns:
    matches = songs_hengameh[col].astype(str).apply(lambda x: bool(emoji_pattern.search(x)))
    matches.count()

In [33]:
songs_hengameh.applymap(lambda x: 'http' in str(x)).sum()

title            0
rank             0
price            0
asin             0
popular_asin     0
product_info    27
dtype: int64

In [32]:
def clean_text(text):
    text = re.sub(r'http\S+|https\S+|www\.\S+|class="[^"]*"', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'<(span|h1)\b', '', text)
    text = re.sub(r'\n+|[^\w\s]+', '', text)
    text = text.strip().lower()
    return text

In [34]:
songs_hengameh['rank'] = songs_hengameh['rank'].fillna(0).astype(str)
songs_hengameh['rank'] = songs_hengameh['rank'].replace(',','')
songs_hengameh['rank'] = songs_hengameh['rank'].str.extract('([\d,]+)')
songs_hengameh['rank'] = songs_hengameh['rank'].str.replace(',', '')
songs_hengameh['rank'] = songs_hengameh['rank'].astype(int)

In [35]:
songs_hengameh['price'] = songs_hengameh['price'].astype(float)

In [36]:
songs_hengameh.columns

Index(['title', 'rank', 'price', 'asin', 'popular_asin', 'product_info'], dtype='object')

In [37]:
text_cols = ['title', 'product_info', 'popular_asin']

for col in text_cols:
    songs_hengameh[col] = songs_hengameh[col].fillna('').apply(lambda x: clean_text(str(x)))

In [38]:
songs_hengameh.applymap(lambda x: 'http' in str(x)).sum()

title           0
rank            0
price           0
asin            0
popular_asin    0
product_info    0
dtype: int64

In [39]:
asin_to_title = dict(zip(songs_hengameh['asin'], songs_hengameh['title']))

def replace_asins(cell):
    asins = cell.split()
    titles = [asin_to_title.get(asin, None) for asin in asins]
    titles = [t for t in titles if t is not None]
    return ' '.join(titles)

songs_hengameh['popular_asin'] = songs_hengameh['popular_asin'].apply(replace_asins)

In [40]:
songs_hengameh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74347 entries, 0 to 74346
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         74347 non-null  object 
 1   rank          74347 non-null  int32  
 2   price         74347 non-null  float64
 3   asin          74347 non-null  object 
 4   popular_asin  74347 non-null  object 
 5   product_info  74347 non-null  object 
dtypes: float64(1), int32(1), object(4)
memory usage: 3.1+ MB


In [41]:
songs_hengameh['product_info'] = songs_hengameh['product_info'].fillna('') + ' ' + songs_hengameh['popular_asin'].fillna('')

In [42]:
songs_hengameh = songs_hengameh.drop(['popular_asin'],axis=1)

In [56]:
songs_hengameh.head(5)

,title,rank,price,asin,product_info
0,master collection volume one,58291,18.99,0001377647,master collection volume one john michael tal...
1,hymns collection hymns 1 amp 2,93164,12.35,0001529145,hymns collection hymns 1 amp 2 second chapter...
2,early works don francisco,875825,12.35,0001527134,early works don francisco don francisco don ...
3,so you wanna go back to egypt,203263,13.01,0001388703,so you wanna go back to egypt keith green
4,early works dallas holm,399269,12.35,0001526146,early works dallas holm 1 losing game 2 i can...


In [44]:
songs_hengameh['product_info'].to_csv('product_info.csv', index=False)

In [45]:
songs_hengameh.shape

(74347, 5)

In [46]:
new_songs_hengameh = pd.read_csv('product_info.csv')
new_songs_hengameh.head(3)

,product_info
0,master collection volume one john michael tal...
1,hymns collection hymns 1 amp 2 second chapter...
2,early works don francisco don francisco don ...


In [47]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(songs_hengameh['product_info'])

In [48]:
feature_names = tfidf.get_feature_names()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [49]:
matrix_row = tfidf_matrix[10]
dense_row = matrix_row.todense()
non_zero_indices = dense_row.nonzero()[1]

for index in non_zero_indices[10:15]:
    word = feature_names[index]
    tfidf_value = dense_row[0, index]
    print(f'{word:{10}} {index:{10}} \t {tfidf_value:.4f}')

elementary      63417 	 0.1303
excellent       66668 	 0.0864
foundation      71859 	 0.0991
ginn            75982 	 0.2950
grade           78067 	 0.1110


In [50]:
tfidf_matrix.shape

(74347, 188250)

In [51]:
# TruncatedSVD: reduce the dimensionality of the TF-IDF
# svd = TruncatedSVD(n_components=200)
# svd_matrix = svd.fit_transform(tfidf_matrix)

In [52]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [53]:
for i in range(5):
    for j in range(i+1, 5):
        score = cosine_sim[i][j]
        print(f"\nSimilarity score between '{songs_hengameh.iloc[i]['title']}' and '{songs_hengameh.iloc[j]['title']}' is {score:.3f}")


Similarity score between 'master collection volume one' and 'hymns collection hymns 1 amp 2' is 0.021

Similarity score between 'master collection volume one' and 'early works  don francisco' is 0.000

Similarity score between 'master collection volume one' and 'so you wanna go back to egypt' is 0.000

Similarity score between 'master collection volume one' and 'early works  dallas holm' is 0.015

Similarity score between 'hymns collection hymns 1 amp 2' and 'early works  don francisco' is 0.000

Similarity score between 'hymns collection hymns 1 amp 2' and 'so you wanna go back to egypt' is 0.000

Similarity score between 'hymns collection hymns 1 amp 2' and 'early works  dallas holm' is 0.000

Similarity score between 'early works  don francisco' and 'so you wanna go back to egypt' is 0.000

Similarity score between 'early works  don francisco' and 'early works  dallas holm' is 0.027

Similarity score between 'so you wanna go back to egypt' and 'early works  dallas holm' is 0.000


In [54]:
def recommend_songs(title, cosine_sim=cosine_sim, songs=songs_hengameh):
    idx = songs[songs['product_info'].str.contains(title)].index[0]
    print(f"\nIf you mean {songs.iloc[idx]['title']} song :\n")
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    song_indices = [i[0] for i in sim_scores]
    return songs['title'].iloc[song_indices]

In [57]:
while True:
    title = input('Enter a song title :').lower()
    if title == 'exit':
        break
    if not any(songs_hengameh['product_info'].str.contains(title)):
        print(f"\nWe don't have recommendations for {title} ")
    else:
        recommended_songs = recommend_songs(title)
        print("\nRecommended songs\n")
        for i, song in enumerate(recommended_songs):
            print(f"{i+1}. {song}")
        print('='*70)

Enter a song title :world

If you mean the life the complete recorded trilogy on the life of christ song :


Recommended songs

1. luke a world turned upside down
2. luke jones  just believe
3. luke powerstexaseeub001f0tq8qrockpop
4. the upside down world of john pantry lp
5. upside down
6. here and now
7. upside of social
8. kill the lights by luke bryan 20150807
9. official bootleg
10. this is the life
Enter a song title :exit
